In [80]:
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp
from statsmodels.stats.proportion import proportion_confint
from functools import partial

from pycno.pycno import pycno, save_pycno, extract_values

% matplotlib inline

In [12]:
# Make a pycnophylactic surface for NYC based on block groups.
# Get NYC shapefile
nyc = gpd.read_file(r'D:\NYC\pop_data\pop_data\bg_shp\nyc_project_32118_popn.shp')
# Run algorithm
nyc_pycno, trans, crs = pycno(nyc,'PopEst_Tot',25)
# Save raster
save_pycno(nyc_pycno, trans,crs,'nyc_pycno.tif')

pycno\pycno.py:138: RuntimeWarning: invalid value encountered in less
  value_array[value_array<0] = 0.0
pycno\pycno.py:114: RuntimeWarning: invalid value encountered in true_divide
  correct = val / nansum(data[mask])
pycno\pycno.py:144: RuntimeWarning: invalid value encountered in absolute
  print("Maximum Change: " + str(round(nanmax(absolute(old - value_array)),4)) + " - will stop at " + str(round(stopper,4)))
pycno\pycno.py:146: RuntimeWarning: invalid value encountered in absolute
  if nanmax(absolute(old - value_array)) < stopper:


Maximum Change: 56.4579 - will stop at 0.1683
Maximum Change: 20.8216 - will stop at 0.1683
Maximum Change: 11.889 - will stop at 0.1683
Maximum Change: 8.1063 - will stop at 0.1683
Maximum Change: 5.3725 - will stop at 0.1683
Maximum Change: 4.461 - will stop at 0.1683
Maximum Change: 3.5476 - will stop at 0.1683
Maximum Change: 3.1437 - will stop at 0.1683
Maximum Change: 2.652 - will stop at 0.1683
Maximum Change: 2.3208 - will stop at 0.1683
Maximum Change: 2.0043 - will stop at 0.1683
Maximum Change: 1.7565 - will stop at 0.1683
Maximum Change: 1.5764 - will stop at 0.1683
Maximum Change: 1.4714 - will stop at 0.1683
Maximum Change: 1.3754 - will stop at 0.1683
Maximum Change: 1.2871 - will stop at 0.1683
Maximum Change: 1.206 - will stop at 0.1683
Maximum Change: 1.1278 - will stop at 0.1683
Maximum Change: 1.0635 - will stop at 0.1683
Maximum Change: 1.0 - will stop at 0.1683
Maximum Change: 0.9322 - will stop at 0.1683
Maximum Change: 0.8713 - will stop at 0.1683
Maximum Change

In [13]:
# DNAInfo Data
ny_data = r'NYC_Analysis_wgs84.geojson'

ny_nhoods = gpd.read_file(ny_data)

# Set the crs of the input geojson to WGS84
ny_nhoods.crs = {'init': 'epsg:4326'}

# Tranform to projected coordinate system - EPSG:32118 - New York Long Island, NAD83-based projection in metres.
ny_nhoods = ny_nhoods.to_crs({'init':'epsg:32118'})

ny_nhoods.head()

,neighborhoodLive,otherNeighborhood,shapeID,yearsLived,nhood,geometry
0,Allerton,,259,2,Allerton,"POLYGON ((311834.6421283115 77460.83003323646,..."
1,Allerton,,298,0,Allerton,"POLYGON ((311305.0904452917 78317.74772848346,..."
2,Allerton,,5964,2,Allerton,"POLYGON ((311189.369271408 78288.74123209271, ..."
3,Allerton,,59957,15,Allerton,"POLYGON ((311199.2696228603 77331.72545915539,..."
4,Allerton,,61660,1,Allerton,"POLYGON ((311137.8304619542 78879.66082988479,..."


In [37]:
# open nyc pycno raster
nyc_pycno = rasterio.open('nyc_pycno.tif', nodata=np.nan)
trans = nyc_pycno.transform
# Extract values (NB must read to Numpy array first)
ny_nhoods_pops = extract_values(nyc_pycno.read()[0],ny_nhoods,trans,'PopEst')
ny_nhoods_pops.head()

,neighborhoodLive,otherNeighborhood,shapeID,yearsLived,nhood,geometry,PopEst
0,Allerton,,259,2,Allerton,"POLYGON ((311834.6421283115 77460.83003323646,...",2734.993117
1,Allerton,,298,0,Allerton,"POLYGON ((311305.0904452917 78317.74772848346,...",4810.774235
2,Allerton,,5964,2,Allerton,"POLYGON ((311189.369271408 78288.74123209271, ...",86478.777494
3,Allerton,,59957,15,Allerton,"POLYGON ((311199.2696228603 77331.72545915539,...",11467.052032
4,Allerton,,61660,1,Allerton,"POLYGON ((311137.8304619542 78879.66082988479,...",39964.221017


In [44]:
# Work out the estimated population density of drawings
ny_nhoods_pops['pop_dense_per_km2'] = ny_nhoods_pops['PopEst']/(ny_nhoods_pops['geometry'].area/1000000)

# Group data by neighborhood
ny_grp = ny_nhoods_pops.groupby('nhood')

In [87]:
# Confidence interval around mean
lci = lambda s: sp.t.interval(0.95, len(s)-1, loc=np.mean(s), scale=sp.sem(s))[0]
uci = lambda s: sp.t.interval(0.95, len(s)-1, loc=np.mean(s), scale=sp.sem(s))[1]

# Confidence interval around median
m_lci = lambda s: np.percentile(s,q=100*proportion_confint(int(s.size/2),int(s.size),0.05,'beta')[0])
m_uci = lambda s: np.percentile(s,q=100*proportion_confint(int(s.size/2),int(s.size),0.05,'beta')[1])

aggs = {'n':np.size,'mean': np.mean,'uci_95': uci, 'lci_95':lci,'median':np.median,'m_lci':m_lci,'m_uci':m_uci,'lq':partial(np.percentile,q=25),'uq':partial(np.percentile,q=75)}

# Get summaries of population size and density
pop_sum = ny_grp['PopEst'].agg(aggs)
pop_dense_sum = ny_grp['pop_dense_per_km2'].agg(aggs)


C:\Users\ppehdlew\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  if sys.path[0] == '':
C:\Users\ppehdlew\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  del sys.path[0]


In [90]:
# save out the tables
pop_sum.to_excel('DNAInfo_Populations.xlsx')
pop_dense_sum.to_excel('DNAInfo_Population_Densities.xlsx')